## Cosas interesantes
- Los cif de las empresas españolas siempre tienen 9 caracteres (letra mas 8 numeros)
- Los dni siempre tienen 8 letras mas un numero
- Los codigos postales españoles siempre son de 5 digitos

## Datos que puedo parsear con re
- cif espanholes
- dni
- codigos postales espanholes
- numero factura
- importe factura
- potencia contratada

# Codigo

In [796]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import PyPDF2
import re
import json
import os
from datetime import datetime

In [182]:
#Separar las palabras que solo pueden contener letras (nombres, localidades...)
#Separar las palabras que solo pueden contener numeros (cp, importes...)
#Separar las palabras que terminan o empiezan por una letra o numero (dni, cif)

In [184]:
train_path = 'training/'

In [186]:
json_paths = [train_path + file for file in os.listdir(train_path) if file.endswith('.json')]
pdf_paths = [train_path + file for file in os.listdir(train_path) if file.endswith('.pdf')]

In [188]:
json_files = {}
for json_path in json_paths:
    bill_id = json_path.removeprefix(train_path + 'factura_').removesuffix('.json')
    bill_id = int(bill_id)
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    json_files[bill_id] = data

In [16]:
with open("pdf_files.json", "r") as file:
    pdf_files = json.load(file)

In [11]:
'''json_files = {}
pdf_files = {}
for json_path, pdf_path in zip(json_paths, pdf_paths):
    json_bill_id = json_path.removeprefix(train_path + 'factura_').removesuffix('.json')
    json_bill_id = int(json_bill_id)
    
    pdf_bill_id = pdf_path.removeprefix(train_path + 'factura_').removesuffix('.pdf')
    pdf_bill_id = int(pdf_bill_id)
    
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    json_files[json_bill_id] = data
    
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
    
        # Obtener el número de páginas
        num_pages = len(reader.pages)

        # Extraer texto de todas las páginas
        all_text = ""
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            all_text += page.extract_text()
        all_text = all_text.replace('\n','')
    pdf_files[pdf_bill_id] = all_text'''

'json_files = {}\npdf_files = {}\nfor json_path, pdf_path in zip(json_paths, pdf_paths):\n    json_bill_id = json_path.removeprefix(train_path + \'factura_\').removesuffix(\'.json\')\n    json_bill_id = int(json_bill_id)\n    \n    pdf_bill_id = pdf_path.removeprefix(train_path + \'factura_\').removesuffix(\'.pdf\')\n    pdf_bill_id = int(pdf_bill_id)\n    \n    with open(json_path, \'r\') as file:\n        data = json.load(file)\n    \n    json_files[json_bill_id] = data\n    \n    with open(pdf_path, \'rb\') as file:\n        reader = PyPDF2.PdfReader(file)\n    \n        # Obtener el número de páginas\n        num_pages = len(reader.pages)\n\n        # Extraer texto de todas las páginas\n        all_text = ""\n        for page_num in range(num_pages):\n            page = reader.pages[page_num]\n            all_text += page.extract_text()\n        all_text = all_text.replace(\'\n\',\'\')\n    pdf_files[pdf_bill_id] = all_text'

## Pruebas re

In [945]:
def extract_result(matches):
    matches_set = set(matches)
    if len(matches_set) == 1:
        return matches[0]
    return matches

In [305]:
num_bills = [0,1,4,47]

In [352]:
'''text_bills = {}
for num in num_bills:
    reader = PyPDF2.PdfReader(f"training/factura_{num}.pdf")

    num_pages = len(reader.pages)

    text = reader.pages[0].extract_text()
    
    text_bills[num] = text'''

'text_bills = {}\nfor num in num_bills:\n    reader = PyPDF2.PdfReader(f"training/factura_{num}.pdf")\n\n    num_pages = len(reader.pages)\n\n    text = reader.pages[0].extract_text()\n    \n    text_bills[num] = text'

In [354]:
text_bills = {}
for num in num_bills:
    text = extract_text(f"training/factura_{num}.pdf")

    text_bills[num] = text

In [750]:
text = text_bills[47]

In [752]:
#importe total

re_importe_total = re.compile(r"(\d+,\d+)[^\n]*€")

matches = re_importe_total.findall(text)
max([float(amount.replace(',','.')) for amount in matches])

132.93

In [930]:
#Numero de factura

re_numero_factura = re.compile(r"(?:nº|num\.|número|num)\s*factura:\s*(\w+)", re.IGNORECASE)

matches = re_numero_factura.findall(text)
matches

['D6034231723']

In [942]:
#cif

re_cif = re.compile(r"\b[A-Z]\d{8}\b")

matches = re_cif.findall(text)
matches

['B02494375']

In [757]:
#dni

re_dni = re.compile(r"\b\d{8}[A-Z]\b")

matches = re_dni.findall(text)
matches

['36368114P']

In [758]:
#cp
#tener cuidado con el resto de datos numericos euros, kwh, kw

re_cp = re.compile(r".*\b\d{5}\b(?!\s*kW|\s*kWh).*")

matches = re_cp.findall(text)
matches

['Remite: INER ENERGIA CASTILLA LA MANCHA SL  Apartado  de  Correos  X X X X X   02006 ALBACETE ',
 'Domicilio social: C/ SANTIAGO RUSIÑOL, 9 BAJO, 02006 - Albacete ',
 '28911 Leganés ',
 'Dirección de suministro: Calle Sonsa 28911 Leganés ',
 'Dirección fiscal: Calle Sonsa 28911  Leganés ',
 'Puntos de atención cercanos: C/ SANTIAGO RUSIÑOL, 9 BAJO 02006 ',
 'Departamento de Reclamaciones, Apdo. de Correos XXXXX – 02006 ALBACETE ']

In [759]:
#potencia contratada

re_potencia_contratada = re.compile(r"contratada:?[^\n]*?(\d+,\d+)\s*(?<!€/)kW")

matches = re_potencia_contratada.findall(text)
matches

['4,70']

In [949]:
#consumo
#Mirar si puede estar en la segunda pagina y decidir si usar \s* o [^\n]*

re_consumo = re.compile(r"(\d+)\s*kWh")

matches = re_consumo.findall(text)
matches = [int(amount.replace(',','.')) for amount in matches]
matches = sorted(matches)
if len(matches) >= 2:
    consumo = matches[-1] - matches [-2]
    if consumo in matches:
        print(consumo) 
else:
    print(matches)

368


In [890]:
# Fechas periodo
# Comprobar por cada elemento de la lista si hay meses en letra, fechas en distintos formatos o numeros
months = {
    "enero": "01",
    "febrero": "02",
    "marzo": "03",
    "abril": "04",
    "mayo": "05",
    "junio": "06",
    "julio": "07",
    "agosto": "08",
    "septiembre": "09",
    "octubre": "10",
    "noviembre": "11",
    "diciembre": "12"
}

#Hacemos match con todos los elementos que van precedidos por periodo
matches = re.findall(r"(?<=periodo|período)(.*?)(?=\n|$)", text, re.IGNORECASE)

#filtramos los elementos que pueden tener formato de fecha
re_numeric_date = re.compile(r"\b(\d{2})[-.\s\/]\s?(\d{2})[-.\s\/]\s?(\d{4})\b")
re_string_date = re.compile(r"\b(\d{1,2})\s*de?\s*?(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s*de?\s*?(\d{4})\b")
period_dates = [re_numeric_date.findall(match) for match in matches if len(re_numeric_date.findall(match)) == 2]
if not period_dates:
    period_dates = [re_string_date.findall(match) for match in matches if len(re_string_date.findall(match)) == 2]
    period_dates = [[f"{date[0]}.{months[date[1]]}.{date[2]}" for dates in period_dates for date in dates]]
else:
    period_dates = [[f"{date[0]}.{date[1]}.{date[2]}" for dates in period_dates for date in dates]]

#Compruebo que los pares de fechas son distintos entre si para poder obtener periodos
period_dates = [dates for dates in period_dates if dates[0] != dates[1]]

#Me quedo con los valores unicos de fechas para comprobar que solo tenga un periodo valido
period_dates = [date for dates in period_dates for date in dates]
period_dates = list(set(period_dates))

#Ahora le doy formato a las fechas para comparar y parsear la mayor a fecha final y la menor a fecha inicial
date_1 = datetime.strptime(period_dates[0], "%d.%m.%Y")
date_2 = datetime.strptime(period_dates[1], "%d.%m.%Y")
if date_1 == date_2:
    #Decidir que pasa si son iguales
    pass
else:
    sorted_dates = sorted([date_1, date_2]) 
    initial_date, final_date = [date.strftime("%d.%m.%Y") for date in sorted_dates]

### Prueba resultado regex

In [958]:
data = []
for num in range(1000):
    text = extract_text(f"training/factura_{num}.pdf")
    
    #importe total

    re_importe_total = re.compile(r"(\d+,\d+)[^\n]*€")

    matches = re_importe_total.findall(text)
    importe_total = max([float(amount.replace(',','.')) for amount in matches])

    #Numero de factura

    re_numero_factura = re.compile(r"(?:nº|num\.|número|num)\s*factura:\s*(\w+)", re.IGNORECASE)

    matches = re_numero_factura.findall(text)
    numero_factura = extract_result(matches)

    #cif

    re_cif = re.compile(r"\b[A-Z]\d{8}\b")

    matches = re_cif.findall(text)
    cif = extract_result(matches)

    #dni

    re_dni = re.compile(r"\b\d{8}[A-Z]\b")

    matches = re_dni.findall(text)
    dni = extract_result(matches)

    #cp
    #tener cuidado con el resto de datos numericos euros, kwh, kw

    re_cp = re.compile(r".*\b(\d{5})\b(?!\s*kW|\s*kWh).*")

    matches = re_cp.findall(text)
    cp = extract_result(matches)

    #potencia contratada

    re_potencia_contratada = re.compile(r"contratada:?[^\n]*?(\d+,\d+)\s*(?<!€/)kW")

    matches = re_potencia_contratada.findall(text)
    potencia_contratada = extract_result(matches)

    #consumo
    #Mirar si puede estar en la segunda pagina y decidir si usar \s* o [^\n]*

    re_consumo = re.compile(r"(\d+)\s*kWh")

    matches = re_consumo.findall(text)
    matches = [int(amount.replace(',','.')) for amount in matches]
    matches = sorted(matches)
    if len(matches) == 1:
        consumo = matches[0]
    elif len(matches) >= 3:
        consumo = matches[-1] - matches [-2]
        if consumo not in matches:
            consumo = matches
    else:
        consumo = matches

    # Fechas periodo
    # Comprobar por cada elemento de la lista si hay meses en letra, fechas en distintos formatos o numeros
    months = {
        "enero": "01",
        "febrero": "02",
        "marzo": "03",
        "abril": "04",
        "mayo": "05",
        "junio": "06",
        "julio": "07",
        "agosto": "08",
        "septiembre": "09",
        "octubre": "10",
        "noviembre": "11",
        "diciembre": "12"
    }

    #Hacemos match con todos los elementos que van precedidos por periodo
    matches = re.findall(r"(?<=periodo|período)(.*?)(?=\n|$)", text, re.IGNORECASE)

    #filtramos los elementos que pueden tener formato de fecha
    re_numeric_date = re.compile(r"\b(\d{2})[-.\s\/]\s?(\d{2})[-.\s\/]\s?(\d{4})\b")
    re_string_date = re.compile(r"\b(\d{1,2})\s*de?\s*?(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s*de?\s*?(\d{4})\b")
    period_dates = [re_numeric_date.findall(match) for match in matches if len(re_numeric_date.findall(match)) == 2]
    if not period_dates:
        period_dates = [re_string_date.findall(match) for match in matches if len(re_string_date.findall(match)) == 2]
        period_dates = [[f"{date[0]}.{months[date[1]]}.{date[2]}" for dates in period_dates for date in dates]]
    else:
        period_dates = [[f"{date[0]}.{date[1]}.{date[2]}" for dates in period_dates for date in dates]]

    #Compruebo que los pares de fechas son distintos entre si para poder obtener periodos
    period_dates = [dates for dates in period_dates if dates[0] != dates[1]]

    #Me quedo con los valores unicos de fechas para comprobar que solo tenga un periodo valido
    period_dates = [date for dates in period_dates for date in dates]
    period_dates = list(set(period_dates))

    #Ahora le doy formato a las fechas para comparar y parsear la mayor a fecha final y la menor a fecha inicial
    date_1 = datetime.strptime(period_dates[0], "%d.%m.%Y")
    date_2 = datetime.strptime(period_dates[1], "%d.%m.%Y")
    if date_1 == date_2:
        #Decidir que pasa si son iguales
        pass
    else:
        sorted_dates = sorted([date_1, date_2]) 
        initial_date, final_date = [date.strftime("%d.%m.%Y") for date in sorted_dates]
        
    row = [num, importe_total, numero_factura, cif, dni, cp, potencia_contratada, consumo, initial_date, final_date]
    data.append(row)

df_pred = pd.DataFrame(data, columns = ["num", "importe_total", "numero_factura", "cif", "dni", "cp", "potencia_contratada", "consumo", "initial_date", "final_date"])

In [959]:
df_pred.to_csv("predicciones.csv", index = False)

In [952]:
data = []

for num in range(1000):
    importe_total = json_files[num]["importe_factura"]
    numero_factura = json_files[num]["número_factura"]
    dni = json_files[num]['dni_cliente']
    cif = json_files[num]['cif_comercializadora']
    cp = json_files[num]["cp_cliente"]
    potencia_contratada = json_files[num]["potencia_contratada"]
    consumo = json_files[num]["consumo_periodo"]
    initial_date = json_files[num]["inicio_periodo"]
    final_date = json_files[num]["fin_periodo"]
    
    row = [num, importe_total, numero_factura, cif, dni, cp, potencia_contratada, consumo, initial_date, final_date]
    data.append(row)
    
df = pd.DataFrame(data, columns = ["num", "importe_total", "numero_factura", "cif", "dni", "cp", "potencia_contratada", "consumo", "initial_date", "final_date"])

In [374]:
text

'F\nI\n\nC\n\n.\n\n0\n3\n1\n\nn\nó\ni\nc\np\ni\nr\nc\ns\nn\ni\n\n,\n\n3\n3\n3\n5\n6\n-\nM\no\nr\ne\nm\nú\nn\n\na\nj\no\nh\n\n,\n\n7\n\nn\nó\ni\nc\nc\ne\ns\n\n,\n\n2\n3\n\no\ni\nl\no\nf\n\n,\n\n8\n9\n9\n8\n3\n\no\nm\no\nt\n\n,\n\nI\n\nD\nR\nD\nA\nM\n\n.\n\nD\n\nI\n\nR\nD\nA\nM\n\n-\n\n4\n3\n0\n8\n2\n\n,\n\n2\n\n,\n\nA\nD\nI\nT\nS\nA\nB\nA\nL\n\n/\n\nC\n\n:\nl\na\ni\nc\no\nS\n\n.\n\nm\no\nD\n\n.\n\nL\nS\n\n.\n\n,\n\nS\nE\nL\nB\nA\nV\nO\nN\nE\nR\nS\nA\nG\nR\nE\nN\nE\nS\nY\nL\n\nI\n\n.\n\n6\n0\n4\n6\n4\n4\n8\n8\nB\n\nLYS ENERGIAS RENOVABLES, S.L \n Fecha de emisión:  13.06.2020  \nNº factura:  K9131055493 \n\nNombre:  LEONARDA JARAMILLO BÁEZ \nDirección suministro:  Camino de los Pelendengues  \n21830 Bonares \nNº Referencia: 673529209761/1194 \nNº  cliente: XXXXXXX \nNIF: 90237166R \nDirección:  Camino de los Pelendengues 21830 Bonares \nEntidad: Banco de Albacete, S.A. \nDatos bancarios: IBAN ES00914408492265477***** \n Fecha de cargo: 16.06.2020 \n\nLEONARDA JARAMILLO BÁEZ \nCamino de l

In [281]:
pip install pdfminer.six

  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Note: you may need to restart the kernel to use updated packages.


# cosas

In [42]:
reader = PyPDF2.PdfReader("training/factura_1.pdf")
    
# Obtener el número de páginas
num_pages = len(reader.pages)

# Extraer texto de todas las páginas
all_text = ""
for page_num in range(num_pages):
    page = reader.pages[page_num]
    all_text += page.extract_text()
print(all_text)

        L...u..n..e..s.. .a.. .s..á..b..a..d..o..,. .d..e.. .8.. .a.. .2..2.. .h..o..r.a..s.......................... 
Contratación Productos y Servicios  XXXXXXX  
Reclamaciones  e incidencias    900 101 022   
 
C/ LABASTIDA, 2, 28034 - MADRID  
Si quiere  una atención  más personalizada  puede  acudir  a alguno  de los centros  que LYS ENERGIAS RENOVABLES, 
S.L tiene  a su  servicio. Encuentre el más cercano en http://enigma -technologies.com/ /centros o desde su móvil en  
http://enigma -technologies.com/ /.mobi.  2..4.. .h..o...r.a..s.. ./. .3..6..5.. .d..í.a..s.. .d..e..l. .a..ñ..o................................... 
Web    http://enigma -technologies.com/  
Reparación  urgente  de calderas  
y electrodomésticos  XXX XXX XXX  
Lectura  del contador  XXX  XXX XXX  
http://enigma -technologies.com/ /lecturas  
Averías  eléctricas  900 101 061    
 
 
  LYS ENERGIAS RENOVABLES, S.L  
 Fecha  de emisión:  13.06.2020   
Nº factura:  K9131055493  
Nombre:  LEONARDA JARAMILLO BÁEZ  
Dir

In [36]:
pdf_files['65']

'DATOS  DE LA FACTURA  Nº factura:  J8985000598  Referencia:  956772786605/7586  Fecha  emisión  factura:  21/10/2009  Periodo  de Facturación:  del 20/08/2009  a 19/10/2009  (60 días)  Fecha  de cargo:  24 de octubre de 2009  ............................................................................................................................. .......................................  CAPITAL ENERGY COMERCIALIZADORA, S.L.U . CIF B88575063 . CALLE MARQUÉS DE VILLAMAGNA Nº 3 - 5ª PLANTA (TORRE SERRANO)  28001  - MADRID  GERALDINE FRÍAS UREÑA   Finca Javega  01015  Vitoria -Gasteiz  Araba/Álava  Forma  de pago:  Domiciliada  Potencia  Energía  Descuentos  Otros  Impuestos  IGIC  reducido  20,42  € 79,90  € -X,XX € 0,93 € 5,13 € 15,82  € Fecha  de cargo:  24 de octubre de 2009  IBAN:  ES01457803845316340*****  Cod.Mandato:  E90255220703511896184926562  ( 15%) Versión:  3695  IGIC  normal  ( 21%) 0,20 € ..................................................................................

In [303]:
re_dni = r"\b\d{8}[A-Z]\b"
re_cif_national = r"\b[A-Z]\d{8}\b"
re_cif_foreign = r"\b[A-Z]{2}\d{6,8}\b"

In [305]:
bill_dni_cif = {}
for bill_id, text in pdf_files.items():
    dni = set(re.findall(re_dni, text))
    dni = list(dni)[0] if len(dni) == 1 else None
    
    cif = set(re.findall(re_cif_national, text))
    
    if len(cif) == 1:
        cif = list(cif)[0]
    else:
        cif = set(re.findall(re_cif_foreign, text))
        cif = list(cif)[0] if len(cif) == 1 else None
    
    bill_dni_cif[bill_id] = [dni, cif]

In [306]:
preds_dic = {}
for i in range(1000):
    pred_dni, pred_cif = bill_dni_cif[i]
    
    dni = json_files[i]['dni_cliente']
    cif = json_files[i]['cif_comercializadora']
    
    preds_dic[i] = [bool(pred_dni == dni), bool(pred_cif == cif)]

In [308]:
df = pd.DataFrame(preds_dic.values(), columns = ['dni', 'cif'])

In [311]:
len([data[1] for data in bill_dni_cif.values() if data[1] == None])

65

In [313]:
sum(df['cif'] == False)

65

In [315]:
df[df['cif'] == False].index

Index([ 46,  90,  99, 109, 124, 139, 142, 148, 164, 201, 240, 283, 296, 367,
       377, 386, 405, 413, 417, 426, 433, 436, 454, 458, 477, 507, 518, 521,
       527, 555, 561, 571, 589, 594, 595, 596, 605, 631, 661, 673, 692, 702,
       713, 724, 728, 742, 765, 774, 780, 789, 796, 855, 861, 874, 884, 900,
       903, 908, 917, 937, 962, 964, 969, 970, 987],
      dtype='int64')

## Dataframe

In [230]:
rows = []
for bill_id, data in json_files.items():
    row = [bill_id] + list(data.values())
    rows.append(row)

columns = ["archivo"] + list(data.keys())
df = pd.DataFrame(data = rows, columns = columns)

In [296]:
df[df['cif_comercializadora'].str.len() != 9]["provincia_comercializadora"].value_counts()

provincia_comercializadora
Extranjero           64
BERLIN (Alemania)     3
Olten (Suiza)         2
Name: count, dtype: int64

In [348]:
df.head(3)

,archivo,nombre_cliente,dni_cliente,calle_cliente,cp_cliente,población_cliente,provincia_cliente,nombre_comercializadora,cif_comercializadora,dirección_comercializadora,cp_comercializadora,población_comercializadora,provincia_comercializadora,número_factura,inicio_periodo,fin_periodo,importe_factura,fecha_cargo,consumo_periodo,potencia_contratada
0,684,KENDRA RAMÍREZ RAMÓN,16260925V,Avenida de la Paloma,34259,Cordovilla la Real,Palencia,"REPSOL COMERCIALIZADORA DE ELECTRICIDAD Y GAS,...",B39540760,"C/ ISABEL TORRES, Nº 19",39011,SANTANDER,Cantabria,Z5231002650,09.05.2016,08.07.2016,"144,31",13.07.2016,426,"5,558"
1,350,FRIDA OLMEDO ESTRELLA,56374095Z,Calle Joaquín Blanco Torrent,46814,la Granja de la Costera,Valencia/Valñncia,CIMA ENERGIA COMERCIALIZADORA SL,B26558056,"AVENIDA DE PORTUGAL, 27 PLANTA 4, PUERTA 8",26001,LOGROÑO,LA RIOJA,QS7674816555,29.12.2011,28.01.2012,"90,57",02.02.2012,313,"2,697"
2,673,Estíbaliz Parra Galván,72326445R,Calle Alejandro Sureda,27520,Carballedo,Lugo,"EXEN, S.R.O.",C227767191,DR. SMERALA 1710/1,70200,OSTRAVA (República Checa),Extranjero,HZ2504915940,07.10.2019,06.11.2019,"11,89",11.11.2019,0,"2,551"


In [358]:
df["nombre_cliente"].str.split().apply(lambda x : len(x)).value_counts()

nombre_cliente
3    952
4     25
5     17
2      6
Name: count, dtype: int64

In [586]:
df["potencia_contratada"].sort_values()

697    0,721
545    0,756
377    1,167
283    1,203
923    1,312
       ...  
523    6,531
485    6,563
581    6,874
176    6,953
360    7,095
Name: potencia_contratada, Length: 1000, dtype: object